In [6]:
import cv2
import numpy as np
from keras.models import load_model

In [7]:
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
mask_model = load_model('ModelWeights/Normal1.h5')

In [9]:
#live video

cap = cv2.VideoCapture(0)
dim = (250,250)
mask_dict = {0: 'No Mask', 1: 'Mask'}
mask_dict_color = {0: (0,0,255), 1: (0,255,0)}
while True: 
    ret, frame = cap.read()
    clone = frame.copy()
    bboxes = classifier.detectMultiScale(clone)
    for i in bboxes: 
        x, y, width, height = i[0], i[1], i[2], i[3]
        x2, y2 = x+ width, y+height
        roi = clone[y:y2, x:x2]
        roi = cv2.resize(roi, dim, interpolation = cv2.INTER_CUBIC)
        roi = roi.reshape(1, roi.shape[0], roi.shape[1], roi.shape[2])
        mask_predict = mask_model.predict(roi)[0]
        predict_idx = np.argmax(mask_predict)
        print(mask_predict)
        cv2.putText(clone, mask_dict[predict_idx], (x, y+15), cv2.FONT_HERSHEY_SIMPLEX, .5, mask_dict_color[predict_idx], 2)
        cv2.rectangle(clone, (x,y), (x2,y2), mask_dict_color[predict_idx], 1)
        
        
    
    
    cv2.imshow('LIVE', clone)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'): 
        break 
cap.release()

cv2.destroyAllWindows()

[1.0000000e+00 1.8290186e-38]
[0. 1.]
[1.04108225e-20 1.00000000e+00]
[0. 1.]
[1.04108225e-20 1.00000000e+00]
[0. 1.]
[2.4668884e-27 1.0000000e+00]
[0. 1.]
[2.4668884e-27 1.0000000e+00]
[0. 1.]
[1.8173347e-29 1.0000000e+00]
[0. 1.]
[1.7114154e-22 1.0000000e+00]
[0. 1.]
[1.7114154e-22 1.0000000e+00]
[0. 1.]
[9.757715e-34 1.000000e+00]
[0. 1.]
[9.757715e-34 1.000000e+00]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[2.506389e-17 1.000000e+00]
[0. 1.]
[2.506389e-17 1.000000e+00]
[0. 1.]
[0.01639173 0.99655116]
[0. 1.]
[0.01639173 0.99655116]
[0. 1.]
[4.9013264e-18 1.0000000e+00]
[0. 1.]
[4.9013264e-18 1.0000000e+00]
[0. 1.]
[1.0000000e+00 4.3185217e-15]
[0. 1.]
[1.0000000e+00 4.3185217e-15]
[0. 1.]
[1.21840886e-32 1.00000000e+00]
[0. 1.]
[1.21840886e-32 1.00000000e+00]
[0. 1.]
[3.9177737e-32 1.0000000e+00]
[0. 1.]
[3.9177737e-32 1.0000000e+00]
[0. 1.]
[0.9976126  0.01532982]
[0. 1.]
[0.9976126  0.01532982]
[0. 1.]
[1.0000000e+00 5.1684772e-23]
[0. 1.]
[1.0000000e+00 5.1684772e-23]
[0. 1.]
[1.9

[0. 1.]
[1.000000e+00 7.876262e-36]
[0. 1.]
[1.000000e+00 2.068573e-34]
[0. 1.]
[1.000000e+00 2.068573e-34]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[4.142775e-19 1.000000e+00]
[1.0000000e+00 2.6343453e-38]
[4.142775e-19 1.000000e+00]
[1.0000000e+00 2.6343453e-38]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1.000000e+00 9.890111e-32]
[0. 1.]
[1.000000e+00 9.890111e-32]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1.0000000e+00 1.4960716e-36]
[0. 1.]
[1.0000000e+00 1.4960716e-36]
[0. 1.]
[1.0000000e+00 1.6114603e-38]
[0. 1.]
[1.0000000e+00 1.6114603e-38]
[4.1672726e-34 1.0000000e+00]
[1.000000e+00 4.325606e-36]
[4.1672726e-34 1.0000000e+00]
[1.000000e+00 4.325606e-36]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1.000000e+00 5.274419e-36]
[0. 1.]
[1.000000e+00 5.274419e-36]
[0. 1.]
[1.0000000e+00 4.897

[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1.0000000e+00 1.8540375e-21]
[0. 1.]
[1.0000000e+00 1.8540375e-21]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1.000000e+00 5.792695e-24]
[0. 1.]
[1.000000e+00 5.792695e-24]
[0. 1.]
[1.000000e+00 2.487621e-37]
[0. 1.]
[1.000000e+00 2.487621e-37]
[0. 1.]
[1.000000e+00 7.502973e-17]
[0. 1.]
[1.000000e+00 7.502973e-17]
[0. 1.]
[1.0000000e+00 3.0098793e-15]
[0. 1.]
[1.0000000e+00 3.0098793e-15]
[0. 1.]
[1.0000000e+00 5.7529013e-22]
[0. 1.]
[1.0000000e+00 5.7529013e-22]
[0. 1.]
[1.000000e+00 3.512816e-30]
[0. 1.]
[1.000000e+00 3.512816e-30]
[0. 1.]
[1.000000e+00 4.626128e-09]
[0. 1.]
[1.000000e+00 4.626128e-09]
[0. 1.]
[1.0000000e+00 1.1020625e-37]
[0. 1.]
[1.0000000e+00 1.10206

In [ ]:
img = cv2.imread('../multiple.png')

In [ ]:
bboxes = classifier.detectMultiScale(img)

In [ ]:
clone = img.copy()
for i in bboxes: 
    x, y, width, height = i[0], i[1], i[2], i[3]
    x2, y2 = x+ width, y+height
    cv2.rectangle(clone, (x,y), (x2,y2), (0,0,255), 1)
    
    
cv2.imshow('test', clone)
cv2.waitKey(0)
cv2.destroyAllWindows()
    